In order to use a classifier for read_state you need a training set of image squares for your game.  See 2023-11-17 - Make a training image set of squares.ipynb to do that.

In this example, my training set is in the folder "images/training squares".

We'll start by debugging, and making sure it works, before making a complete read_state function

In [1]:
from classy import *
from Game import Board

Version  1.0.7
Version:  0.0.46
Version:  0.2.46


In [2]:
images=image.load_images('images/training squares/')
images=remap_targets(images,new_target_names=['blank','player1','player2'])
summary(images)

[player1]: 58 files found
[blank]: 46 files found
[player2]: 56 files found
Images
160 images of shape (50, 50, 3)
Target values given.
Target names: 'blank', 'player1', 'player2'
[blank]: 46 files
[player1]: 58 files
[player2]: 56 files


In [3]:
data=image.images_to_vectors(images)

160 vectors of length 7500
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p7495', 'p7496', 'p7497', 'p7498', 'p7499'  (7500 features)
Target values given.
Target names: 'blank', 'player1', 'player2'
Mean:  [137.15625 144.91875 138.9875  ... 182.66875 186.55    177.63125]
Median:  [153.  161.5 156.  ... 184.  188.  180. ]
Stddev:  [49.87027507 52.84611763 53.1624853  ... 12.93334927 17.7122133
 16.97741952]


The following should really be 100% -- if it can't identify the data that it knows about, it will make errors on any test

In [4]:
C=NaiveBayes()
C.fit(data.vectors,data.targets)
print("On the full data set:",C.percent_correct(data.vectors,data.targets))

On the full data set: 100.0


In [5]:
C.save("naive_bayes_training_squares_trained.json")
C.save("naive_bayes_training_squares_trained2.json")

In [6]:
C1=NaiveBayes()
C1.load("naive_bayes_training_squares_trained.json")
print("On the full data set:",C1.percent_correct(data.vectors,data.targets))

On the full data set: 100.0


In [7]:
C=kNearestNeighbor()
C.fit(data.vectors,data.targets)
print("On the full data set:",C.percent_correct(data.vectors,data.targets))

On the full data set: 99.375


In [8]:
C.save("kNearestNeighbor_training_squares_trained.json")
C.save("kNearestNeighbor_training_squares_trained2.json")

In [9]:
C1=kNearestNeighbor()
C1.load("kNearestNeighbor_training_squares_trained.json")
print("On the full data set:",C1.percent_correct(data.vectors,data.targets))

On the full data set: 99.375


In [10]:
from pylab import imread
from numpy import atleast_2d

In [11]:
fname='images/board to reconstruct - was test9.jpg'
im=imread(fname)

slice into squares

In [12]:
square_size=50 # choose a size that works for you
import json
with open('locations.json') as json_file:
    locations = json.load(json_file)

count=0
im=imread(fname)

values=[]
for r,c in locations:
    sr=r-square_size//2
    er=sr+square_size
    sc=c-square_size//2
    ec=sc+square_size   
    subimage=im[sr:er,sc:ec,:]

    # convert the square image to a data vector for the classifier
    vector=subimage.ravel()
    prediction=C.predict(atleast_2d(vector))[0]

    values.append(prediction)

    count+=1

In [13]:
state=Board(4,4)
state.board=values
state

 1  1  2  1 
 1  2  1  1 
 0  2  0  2 
 1  0  1  1 

## Now put it all together into read_state
    

In [14]:
def read_state_from_file(filename):
    from Game import Board
    text=open(filename).read()
    text=text.strip()
    lines=[line.strip() for line in text.split('\n')]  # get rid of \n
    
    row=lines[0].split()
    R,C=len(lines),len(row)
    print(f"{R}x{C} board")
    state=Board(R,C)
    state.board=[int(val) for val in text.split()]  
    print(state)
    return state

In [15]:
def take_picture(fname):  # in jupyter have this, but don't put this on your robot!
    pass

In [16]:
def read_state():
    from pylab import imread,imsave
    from numpy import atleast_2d
    import os
    from Game import Board
    from classy import image,NaiveBayes

    # train the classifier
    images=image.load_images('images/training squares/',delete_alpha=True)  #<=========
    data=image.images_to_vectors(images,verbose=True)  # train on all of them

    #classifier=kNearestNeighbor()
    classifier=NaiveBayes()
    classifier.load("naive_bayes_training_squares_trained.json")
    #classifier.fit(data.vectors,data.targets)


    # get the picture
    fname='current_board.jpg'              # for the robot
    fname='images/board to reconstruct - was test9.jpg' # for debugging in jupyter
    take_picture(fname)
    im=imread(fname)

    # slice the picture into squares of the right size
    square_size=50 # choose a size that works for you
    import json
    with open('locations.json') as json_file:
        locations = json.load(json_file)

    count=0
    # for debugging
    if not os.path.exists('images/predicted'):
        os.mkdir('images/predicted')
    
    values=[]
    for r,c in locations:
        sr=r-square_size//2
        er=sr+square_size
        sc=c-square_size//2
        ec=sc+square_size   
        subimage=im[sr:er,sc:ec,:]
    
        # convert the square image to a data vector for the classifier
        vector=subimage.ravel()
        prediction=C.predict(atleast_2d(vector))[0]
    
        values.append(prediction)

        # for debugging
        imsave('images/predicted/square %d predicted as %s.jpg' % (count,data.target_names[prediction]),subimage)
    
        count+=1

    
    # reconstruct the state from the predictions
    state=Board(4,4)                                      #<========= change the size
    state.board=values

    print("Current state is:")
    print(state)

    x=input("""
    Hit return if this is correct, otherwise type a character 
    and the state will be read from current_board.txt.""")

    if x:
        print("Reading from file...")
        state=read_state_from_file('board.txt')

    print("Using")
    print(state)

    
    return state


In [17]:
state=read_state()

[player1]: 58 files found
[blank]: 46 files found
[player2]: 56 files found
160 vectors of length 7500
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p7495', 'p7496', 'p7497', 'p7498', 'p7499'  (7500 features)
Target values given.
Target names: 'player1', 'blank', 'player2'
Mean:  [137.15625 144.91875 138.9875  ... 182.66875 186.55    177.63125]
Median:  [153.  161.5 156.  ... 184.  188.  180. ]
Stddev:  [49.87027507 52.84611763 53.1624853  ... 12.93334927 17.7122133
 16.97741952]
Current state is:
 1  1  2  1 
 1  2  1  1 
 0  2  0  2 
 1  0  1  1 




    Hit return if this is correct, otherwise type a character 
    and the state will be read from current_board.txt. 


Using
 1  1  2  1 
 1  2  1  1 
 0  2  0  2 
 1  0  1  1 

